This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [11]:
# Note: I stored the API key in another file that wasn't added to the git repository, and read it from that file.
with open('apikey.txt','r') as file:
    API_KEY = file.read()

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [12]:
# First, import the relevant modules
import requests
from collections import namedtuple, OrderedDict

In [13]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
parameters={'api_key': API_KEY, "limit":1}
r=requests.get("https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json",params=parameters)
r.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Volume',
   'Ex-Dividend',
   'Split Ratio',
   'Adj. Open',
   'Adj. High',
   'Adj. Low',
   'Adj. Close',
   'Adj. Volume'],
  'start_date': '2012-05-18',
  'end_date': '2018-03-27',
  'frequency': 'daily',
  'data': [['2018-03-27',
    156.31,
    162.85,
    150.75,
    152.19,
    76787884.0,
    0.0,
    1.0,
    156.31,
    162.85,
    150.75,
    152.19,
    76787884.0]],
  'collapse': None,
  'order': None}}

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()['dataset_data']

{'limit': 1,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Volume',
  'Ex-Dividend',
  'Split Ratio',
  'Adj. Open',
  'Adj. High',
  'Adj. Low',
  'Adj. Close',
  'Adj. Volume'],
 'start_date': '2012-05-18',
 'end_date': '2018-03-27',
 'frequency': 'daily',
 'data': [['2018-03-27',
   156.31,
   162.85,
   150.75,
   152.19,
   76787884.0,
   0.0,
   1.0,
   156.31,
   162.85,
   150.75,
   152.19,
   76787884.0]],
 'collapse': None,
 'order': None}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Task 1: Collect data from FSE for AFX_X for the entire year of 2017.

In [5]:
# Parameters to pass to requests.get(); the api_key, the start and end dates, and the order of results
parameters={'api_key': API_KEY, "start_date": '2017-01-01', "end_date": '2017-12-31', 'order': 'asc'}
# Send get request to quandl's api url using parameters
r=requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json",params=parameters)

Task 2: Convert the returned JSON object into a Python dictionary.

In [6]:
#convert json object to python dictionary
json_dict=r.json()['dataset_data']

In [7]:
#get column names
column_names=json_dict['column_names']
#view column names
column_names

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

The columns we need are 'Date', 'Open', 'High', 'Low', 'Close', and 'Traded Volume'.

In [8]:
# get list of lists of per-day stock information. 
data=json_dict['data']
# print last 3 rows
data[-3:]

[['2017-12-27',
  51.45,
  51.89,
  50.76,
  51.82,
  None,
  57452.0,
  2957018.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None],
 ['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None]]

In [9]:
# Store the data as a list of namedtuple instances (one per day of info, with fields for date, 
# opening price, daily high, daily low, closing price, and daily trading volume)
StockInfo=namedtuple('StockInfo',['Date','Open','High','Low','Close','Traded_Volume'])
days_list=[]
for day in data:
    days_list.append(StockInfo(day[0],day[1],day[2],day[3],day[4],day[6]))

Task 3: Calculate what the highest and lowest opening prices were for the stock in this period.

In [25]:
# Use list comprehensions to find the max and min of day.Open, filtering out None values.
print("The highest opening price for the stock is $" + 
      str(max(day.Open for day in days_list if day.Open is not None))+".")
print("The lowest opening price for the stock is $" + 
      str(min(day.Open for day in days_list if day.Open is not None))+"0.")

The highest opening price for the stock is $53.11.
The lowest opening price for the stock is $34.00.


Task 4: What was the largest change in any one day (based on High and Low price)?

In [26]:
# List comprehension to find the biggest one-day change.
print("The largest change in any one day was $" + str(round(max(day.High-day.Low for day in days_list),2))+".")

The largest change in any one day was $2.81.


Task 5: What was the largest change between any two days (based on Closing Price)?

In [18]:
# The Quandl API has a parameter 'transform' that lets us calculate the day-by-day change in any of the 
# numeric columns, so we pull the change in the 5th column 'Close'. 
parameters={'api_key': API_KEY, 
            "start_date": '2017-01-01', 
            "end_date": '2017-12-31', 
            'order': 'asc', 
            'transform':'diff',
            'column_index':4}
r2=requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json",params=parameters)
# We then find the maximum absolute value of change for that column using list comprehension. 
print("The largest change between any two days was $" +
      str(max((abs(i[1])) for i in r2.json()['dataset_data']['data']))+".")

The largest change between any two days was $2.56.


6. What was the average daily trading volume during this year?

In [21]:
# Sum together the Traded_Volume values found in each StockInfo namedtuple, 
# then divide by the length of the list of namedtuples
print("The average daily trading volume during this year was $" +
      str(sum(day.Traded_Volume for day in days_list)/len(days_list)))

The average daily trading volume during this year was $89124.33725490196


7. What was the median trading volume during this year?

In [24]:
# A function that calculates the median value for any column in the time-series data.
# daylist is a list of StockInfo namedtuple objects
# fieldname is the column we want to find the median value for
def median_val(daylist, fieldname):
    list_to_sort=[]
    index_num=daylist[0]._fields.index(fieldname) #find the index number of 'fieldname'
    for d in daylist: # creates the list of values for the specified fieldname
        if d[index_num] is not None:
            list_to_sort.append(d[index_num]) 
    list_to_sort.sort() # sorts the list of values
    if len(list_to_sort)%2==1: # when length is odd, finds the value in the middle of the list and returns it
        med_ind=len(list_to_sort)//2+1
        return list_to_sort[med_ind]
    else: # when length is even, averages the two values in the middle of the list
        med_ind=len(list_to_sort)//2
        med_val=(list_to_sort[med_ind-1]+list_to_sort[med_ind])/2
        return med_val
print("The median trading volume during this year was $"+
      str(median_val(days_list, 'Traded_Volume'))+"0.")


The median trading volume during this year was $76600.00.
